In [1]:
import os
import pandas as pd
import numpy as np
import catboost as cat
from lightgbm.callback import early_stopping
from sklearn.model_selection import KFold, GroupKFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import warnings

import math

In [2]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
warnings.filterwarnings('ignore')

path = os.getcwd()

In [3]:
train = pd.read_csv('../input/kddbr2022apk/train_sequence_fold.csv')
test = pd.read_csv('../input/kddbr2022apk/test_sequence.csv')
df = pd.concat([train, test])

features_1 = pd.read_csv('../input/kddbr2022apk/complete_04.csv')
features_1 = features_1[["Filename","w_std","w_mean","w_median","w_min","w_max","w_var","n_std","n_mean","n_median","n_min","n_max","n_var","d_std","d_mean","d_median","d_min","d_max","d_var"]]
features_1.columns = ["Filename","_w_std","_w_mean","_w_median","_w_min","_w_max","_w_var","_n_std","_n_mean","_n_median","_n_min","_n_max","_n_var","_d_std","_d_mean","_d_median","_d_min","_d_max","_d_var"]
features_2 = pd.read_csv('../input/kddbr2022apk/complete_05.csv')
features_3 = pd.read_csv('../input/kddbr2022apk/complete_06.csv')
features_4 = pd.read_csv('../input/kddbr2022apk/complete_12.csv')

features = pd.merge(features_1, features_2, on='Filename', how='left')
features = pd.merge(features, features_3, on='Filename', how='left')


mario_features_tr = pd.read_csv('../input/kddbr2022apk/mario_train_vec_v1.csv')
mario_features_ts = pd.read_csv('../input/kddbr2022apk/mario_test_vec_v1.csv')

mario_features = pd.concat([mario_features_tr, mario_features_ts])
mario_features2 = pd.read_csv('../input/kddbr2022apk/vecs_v3_fe.csv')


In [4]:
df = pd.merge(df, features, on='Filename', how='left')
df = pd.merge(df, mario_features, on='Filename', how='left')
df = pd.merge(df, mario_features2, on='Filename', how='left')
df = pd.merge(df, features_4, on='Filename', how='left')

df = df.sort_values(by=['run_id','run_seq_id'], ascending=False).reset_index(drop=True)

In [5]:
import gc
del features, mario_features, mario_features2, features_4, features_2, features_3, features_1
gc.collect()

23

In [6]:
df

,Filename,Altitude,Delta,North,East,run_id,run_seq_id,fold,_w_std,_w_mean,_w_median,_w_min,_w_max,_w_var,_n_std,_n_mean,_n_median,_n_min,_n_max,_n_var,_d_std,_d_mean,_d_median,_d_min,_d_max,_d_var,base_05_f05_W_0,base_05_f05_W_3,base_05_f05_W_6,base_05_f05_W_9,base_05_f05_W_12,base_05_f05_W_15,base_05_f05_W_18,base_05_f05_W_21,base_05_f05_W_24,base_05_w_std,base_05_w_mean,base_05_w_median,base_05_w_min,base_05_w_max,base_05_w_var,base_05_f05_H_1,base_05_f05_H_4,base_05_f05_H_7,base_05_f05_H_10,base_05_f05_H_13,base_05_f05_H_16,base_05_f05_H_19,base_05_f05_H_22,base_05_f05_H_25,base_05_n_std,base_05_n_mean,base_05_n_median,base_05_n_min,base_05_n_max,base_05_n_var,base_05_f05_D_2,base_05_f05_D_5,base_05_f05_D_8,base_05_f05_D_11,base_05_f05_D_14,base_05_f05_D_17,base_05_f05_D_20,base_05_f05_D_23,base_05_f05_D_26,base_05_d_std,base_05_d_mean,base_05_d_median,base_05_d_min,base_05_d_max,base_05_d_var,base_06_f06_W_0,base_06_f06_W_3,base_06_f06_W_6,base_06_f06_W_9,base_06_f06_W_12,base_06_f06_W_15,base_06_f06_W_18,base_06_f06_W_21,base_06_f06_W_24,base_06_w_std,base_06_w_mean,base_06_w_median,base_06_w_min,base_06_w_max,base_06_w_var,base_06_f06_H_1,base_06_f06_H_4,base_06_f06_H_7,base_06_f06_H_10,base_06_f06_H_13,base_06_f06_H_16,base_06_f06_H_19,base_06_f06_H_22,base_06_f06_H_25,base_06_n_std,base_06_n_mean,base_06_n_median,base_06_n_min,base_06_n_max,base_06_n_var,base_06_f06_D_2,base_06_f06_D_5,base_06_f06_D_8,base_06_f06_D_11,base_06_f06_D_14,base_06_f06_D_17,base_06_f06_D_20,base_06_f06_D_23,base_06_f06_D_26,base_06_d_std,base_06_d_mean,base_06_d_median,base_06_d_min,base_06_d_max,base_06_d_var,x,y,best_dist,dist,x1_mean,x1_std,x1_min,x1_max,y1_mean,y1_std,y1_min,y1_max,x2_mean,x2_std,x2_min,x2_max,y2_mean,y2_std,y2_min,y2_max,d1_mean,d1_std,d1_min,d1_max,d2_mean,d2_std,d2_min,d2_max,x_mean,x_std,x_min,x_max,y_mean,y_std,y_min,y_max,res_mean,res_std,res_min,res_max,d_ratio_mean,d_ratio_std,d_ratio_min,d_ratio_max,x_ratio_mean,x_ratio_std,x_ratio_min,x_ratio_max,y_ratio_mean,y_ratio_std,y_ratio_min,y_ratio_max,base_12_x,base_12_y,base_12_distance,base_12_correlation,base_12_relevance,base_12_confidence
0,ff76ffce183c11f84489bbd0a34a2fcf.jpg,194.801300,-0.043518,NaN,NaN,1662.0,0.0,NaN,0.448201,-3.723183,-4,-4,-3,0.200884,0.413599,2.217993,2,2,3,0.171064,6.984760,62.102365,61.106464,48.836462,81.871851,48.786874,-4,-4,-4,-4,-4,-4,-3,-4,-4,0.333333,-3.888889,-4,-4,-3,0.111111,2,2,2,2,2,2,2,2,2,0.000000,2.000000,2,2,2,0.000000,144.744603,147.271857,158.028478,148.448644,153.450318,163.042939,150.632002,156.000000,164.109719,6.870180,153.969840,153.450318,144.744603,164.109719,47.199377,-4,-4,-4,-4,-4,-4,-3,-4,-4,0.333333,-3.888889,-4,-4,-3,0.111111,2,2,2,2,2,2,2,2,2,0.000000,2.000000,2,2,2,0.000000,0.996004,0.996021,0.995697,0.995821,0.995713,0.995429,0.995756,0.995642,0.995419,0.000214,0.995723,0.995713,0.995419,0.996021,4.559452e-08,-2,-4,400.382317,1095.6364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,2.0,65.230361,0.994240,0.0,0.0
1,feac1b902c7487dcbdf298bd4be75690.jpg,210.220703,-0.028854,NaN,NaN,1661.0,0.0,NaN,0.497181,-1.560554,-2,-2,-1,0.247189,0.411210,4.785467,5,4,5,0.169094,27.250409,116.948627,113.397531,71.105555,188.209989,742.584793,-1,-2,-2,-1,-2,-2,-1,-1,-2,0.527046,-1.555556,-2,-2,-1,0.277778,5,5,5,5,5,5,5,5,5,0.000000,5.000000,5,5,5,0.000000,326.413848,282.863925,257.033072,353.802205,318.007862,276.483273,383.669128,342.077477,302.170482,40.509744,315.835697,318.007862,257.033072,383.669128,1641.039322,-1,-2,-2,-1,-2,-2,-1,-1,-2,0.527046,-1.555556,-2,-2,-1,0.277778,5,5,5,5,5,5,5,5,5,0.000000,5.000000,5,5,5,0.000000,0.977671,0.982806,0.985575,0.974675,0.977969,0.983274,0.969818,0.974281,0.980065,0.005032,0.978459,0.977969,0.969818,0.985575,2.532570e-05,-4,-1,919.695602,2607.0115,63.206574,34.608434,9.735010,110.121971,35.517882,14.163925,11.360019,75.392853,64.6

In [7]:
df['sqrt_altitude'] = np.sqrt(df['Altitude'])
df['sqrt_delta'] = np.sqrt(df['Delta'])
df['sqrt_altitude_delta'] = np.sqrt(df['Altitude'] + df['Delta'])

df['pow2_altitude'] = (df['Altitude'] ** 2)
df['pow2_delta'] = (df['Delta'] ** 2)
df['pow2_altitude_delta'] = (df['Altitude'] + df['Delta']) ** 2

df['log_Altitude'] = np.log(df['Altitude'])
df['log_Delta'] = np.log(df['Delta'])
df['log_base_05_n_mean'] = np.log(df['base_05_n_mean'])

df['pow13_Altitude'] = df['Altitude'] ** (1/3)
df['pow13_Delta'] = df['Delta'] ** (1/3)
df['pow13_base_05_n_mean'] = df['base_05_n_mean'] ** (1/3)

tgt_columns = [
    'Delta', 'Altitude', 'log_Altitude', 'log_Delta', 'pow13_Altitude', 'pow13_Delta', 'log_base_05_n_mean', 'pow13_base_05_n_mean',
    'base_05_n_mean', '_n_mean', 'base_05_f05_H_1', 'base_05_n_median', 'base_05_n_max', 'base_05_n_min', 'base_05_f05_H_4', 'x', 'base_05_f05_H_25', 'base_05_f05_H_7', 'x_mean', 'x1_mean', 'x2_mean',
    'base_05_w_mean', '_w_mean', 'base_05_f05_W_21', 'base_05_w_median', 'base_05_w_max', 'base_05_w_min', 'base_05_f05_W_18', 'y', 'base_05_f05_W_6', 'base_05_f05_W_0', 'y_mean', 'y1_mean', 'y2_mean',
    'base_06_n_mean', 'base_06_f06_H_1', 'base_06_n_median', 'base_06_n_max',
    'base_06_w_mean', 'base_06_f06_W_21', 'base_06_w_median', 'base_06_w_max'
]

seq_win = [2,3,5,10,25,30,50]
nxt_sft_win = [1,2,3,4,5]

for col in tqdm(tgt_columns):
    for win in nxt_sft_win:
        df['shift'+str(win)+'_'+col] = df.groupby('run_id')[col].shift(win).reset_index(0,drop=True)
        df['seq_diff_last'+str(win)+'_'+col] = df.groupby('run_id')[col].diff(win).reset_index(0,drop=True)
    
    for win in seq_win:
        df['seq_avg_w'+str(win)+'_'+col] = df.groupby('run_id')[col].rolling(win).mean().reset_index(0,drop=True)
        df['seq_std_w'+str(win)+'_'+col] = df.groupby('run_id')[col].rolling(win).std().reset_index(0,drop=True)
        df['seq_max_w'+str(win)+'_'+col] = df.groupby('run_id')[col].rolling(win).max().reset_index(0,drop=True)
        df['seq_min_w'+str(win)+'_'+col] = df.groupby('run_id')[col].rolling(win).min().reset_index(0,drop=True)
        df['seq_skw_w'+str(win)+'_'+col] = df.groupby('run_id')[col].rolling(win).skew().reset_index(0,drop=True)
    
    
    df['seq_div_last1_'+col] = df[col] / df['shift1_'+col]
    df['seq_div_last2_'+col] = df[col] / df['shift2_'+col]
    df['seq_div_last3_'+col] = df[col] / df['shift3_'+col]
    df['seq_div_last1_last2_'+col] = df['shift1_'+col] / df['shift2_'+col]
    df['seq_div_last2_last3_'+col] = df['shift2_'+col] / df['shift3_'+col]
    
    df['seq_divdiv_last1-last1_last2_'+col] = df['seq_div_last1_'+col] / df['seq_div_last1_last2_'+col]
    df['seq_divdiv_last1_last2-last2_last3_'+col] = df['seq_div_last1_last2_'+col] / df['seq_div_last2_last3_'+col]
    
    df['seq_min_'+col] = df.groupby('run_id')[col].transform('min')
    df['seq_max_'+col] = df.groupby('run_id')[col].transform('max')
    df['seq_std_'+col] = df.groupby('run_id')[col].transform('std')
    df['seq_avg_'+col] = df.groupby('run_id')[col].transform('mean')
    df['seq_skw_'+col] = df.groupby('run_id')[col].transform('skew')

df['r_vetorial_now'] = (df['base_05_n_mean'] ** 2) + (df['base_05_w_mean'] ** 2)
df['shift_r_vetorial'] = (df['shift1_base_05_n_mean'] ** 2) + (df['shift1_base_05_w_mean'] ** 2)
df['shift2_r_vetorial'] = (df['shift2_base_05_n_mean'] ** 2) + (df['shift2_base_05_w_mean'] ** 2)
df['avg_shift_r_vet'] = (df['r_vetorial_now'] + df['shift_r_vetorial'] + df['shift2_r_vetorial']) / 3
df['div_r_now_shift'] = df['r_vetorial_now'] / df['shift_r_vetorial']
df['div_r_shift1_shift2'] = df['shift_r_vetorial'] / df['shift2_r_vetorial']
df['avg_div_r'] = (df['div_r_now_shift'] + df['div_r_shift1_shift2']) / 2

100%|██████████| 42/42 [03:49<00:00,  5.46s/it]


In [8]:
print(df.shape)

(146262, 2587)


In [9]:
train = df.loc[~df.North.isnull(), :].sort_values(by=['run_id', 'run_seq_id']).reset_index(drop=True)
test = df.loc[df.North.isnull(), :].sort_values(by=['run_id', 'run_seq_id']).reset_index(drop=True)

In [10]:
import gc
del df
gc.collect()

23

In [11]:
train_columns = [col for col in train.columns if col not in ['Filename', 'North', 'East', 'fold', 'run_id', 'run_seq_id']]

In [12]:
n_folds = 5

north_oof_preds = np.zeros(len(train))
east_oof_preds = np.zeros(len(train))

north_test_preds = np.zeros(len(test))
east_test_preds = np.zeros(len(test))

targets = ['North', 'East']

In [13]:
train = train.replace([np.inf, -np.inf], np.nan)
#trn_x = trn_x.fillna(-999)
test = test.replace([np.inf, -np.inf], np.nan)

In [14]:
%%time
for target in targets:
    y = train[target]
    oof_preds = np.zeros(len(train))
    test_preds_fold = np.zeros(len(test))
    score_folds  = []

    # folds = GroupKFold(n_splits=n_folds)
    # for fold_, (trn_, val_) in enumerate(folds.split(y, y, train.run_id)):
    for fold_ in train.fold.sort_values().unique():
        print("Fold: {}".format(fold_))

        trn_ = train.loc[train.fold != fold_].index
        val_ = train.loc[train.fold == fold_].index

        trn_x, trn_y = train.loc[trn_, train_columns], y.loc[trn_]
        val_x, val_y = train.loc[val_, train_columns], y.loc[val_]

        dtrain = cat.Pool(trn_x, trn_y)
        dvalid = cat.Pool(val_x, val_y)
        
        params = {
            'loss_function':       'RMSE',
            'eval_metric':         'RMSE',
            'bootstrap_type': 'Bernoulli', 
            'learning_rate':         0.05,
            #'subsample':              0.9,
            'depth':                   10,
            #'min_data_in_leaf':        50,
            'verbose':               200,
            'task_type':            'GPU',
            'random_seed':             1467
        }
        
        execs = 1
        preds = np.zeros(len(val_x))
        test_preds_exec = np.zeros(len(test))
        
        for p in range(0,execs):
            print("Iteration: {}".format(p+1))
            params['random_seed'] += p
            model = cat.train(params = params,
                              pool = dtrain,
                              eval_set = dvalid,
                              iterations = 5000,
                              early_stopping_rounds = 200)
                            #early_stopping_rounds = 1000)
            del dtrain, dvalid, trn_x
            gc.collect()
            preds += ((model.predict((val_x))) / execs)
            test_preds_exec += ((model.predict((test[train_columns]))) / execs)

        test_preds_fold += (test_preds_exec / n_folds)
        oof_preds[val_] = preds
        score_folds.append(np.sqrt(mean_squared_error(val_y, preds)))
        print("FOLD RMSE = {}".format(np.sqrt(mean_squared_error(val_y, preds))))
        del model, val_x
        gc.collect()

    print("############################################################")
    print("{} - MEAN RMSE = {}".format(target, np.mean(score_folds)))
    print("{} - OOF RMSE = {}".format(target, np.sqrt(mean_squared_error(y, oof_preds))))
    print(score_folds)

    if target == 'North':
        north_oof_preds = oof_preds
        north_test_preds = test_preds_fold
    else:
        east_oof_preds = oof_preds
        east_test_preds = test_preds_fold

Fold: 1.0
Iteration: 1
0:	learn: 1.1376450	test: 1.1491841	best: 1.1491841 (0)	total: 149ms	remaining: 12m 24s
200:	learn: 0.1763200	test: 0.3033435	best: 0.3033435 (200)	total: 25.8s	remaining: 10m 16s
400:	learn: 0.1135626	test: 0.2922512	best: 0.2922453 (399)	total: 51.4s	remaining: 9m 49s
600:	learn: 0.0836702	test: 0.2887902	best: 0.2887363 (598)	total: 1m 17s	remaining: 9m 23s
800:	learn: 0.0656996	test: 0.2870521	best: 0.2870521 (800)	total: 1m 42s	remaining: 8m 58s
1000:	learn: 0.0537430	test: 0.2860176	best: 0.2860176 (1000)	total: 2m 9s	remaining: 8m 35s
1200:	learn: 0.0456255	test: 0.2852798	best: 0.2852798 (1200)	total: 2m 35s	remaining: 8m 11s
1400:	learn: 0.0394999	test: 0.2848344	best: 0.2848344 (1400)	total: 3m 2s	remaining: 7m 48s
1600:	learn: 0.0347433	test: 0.2846298	best: 0.2846298 (1600)	total: 3m 29s	remaining: 7m 24s
1800:	learn: 0.0309237	test: 0.2845278	best: 0.2845083 (1761)	total: 3m 56s	remaining: 7m
2000:	learn: 0.0277098	test: 0.2844022	best: 0.2843907 (19

In [15]:
all_preds = np.concatenate([north_oof_preds, east_oof_preds])
all_target = np.concatenate([train.North, train.East])

In [16]:
print("ALL - OOF RMSE = {}".format(np.sqrt(mean_squared_error(all_target, all_preds))))

ALL - OOF RMSE = 0.3156522622126336


In [17]:
sub_oof = pd.DataFrame({
    'Filename': train.Filename,
    'oof_north': north_oof_preds,
    'oof_east': east_oof_preds
})

sub_oof.to_csv('oof_cat_seq_003.csv', index=False)

In [18]:
sub_north_test = pd.DataFrame({
    'Id': test.Filename,
    'Predicted': north_test_preds
})

sub_east_test = pd.DataFrame({
    'Id': test.Filename,
    'Predicted': east_test_preds
})

In [19]:
sub_north_test.Id = sub_north_test['Id'].astype(str) + ':North'
sub_east_test.Id  = sub_east_test['Id'].astype(str) + ':East'

sub_all_test = pd.concat([sub_north_test, sub_east_test])

In [20]:
sub_all_test[['Id', 'Predicted']].to_csv('sub_cat_seq_003.csv', index=False)

In [21]:
sub_all_test

,Id,Predicted
0,0f57a863af3b7e5bf59a94319a408ff7.jpg:North,-0.406773
1,b7d4e3a6acb3b956d8b09416958e8892.jpg:North,-0.518361
2,9a1bd630a47dabafa6f4cab7338d01df.jpg:North,-0.554890
3,3f6ebf8aef0f352720bc0cb534d878b6.jpg:North,-0.610314
4,7e810f05bd88ff0f86dce7116b9fa527.jpg:North,-0.566292
...,...,...
55026,fc65c23ae61ce2c6fdbc637a8296d1d7.jpg:East,0.757018
55027,fc6c0d99ec42bad5e2d5fbb1e71c5a50.jpg:East,1.093656
55028,fdbb732633c6a3d32a67fc56edc7cde4.jpg:East,-0.422340
55029,feac1b902c7487dcbdf298bd4be75690.jpg:East,-0.468506
